### Does a long context lead to increased hallucinations in LLM's?
In this hackathon, we will investigate if it is possible to detect hallucinations in LLM's given a long context.
<br>In this case, we define an LLM to be hallucinating if the output is unfaithful to the given information.
<br>Your task is to: <br>1) Create a model agnostic solution that can detect any hallucination(s) in the answer that deviate from the context. <br>2) Provide research insights on the problem and lesson's learnt from this hackathon. <br>This is a starter kit for you to start tinkering with the models and to observe it's outputs

In [1]:
import os
import json
import time
from pathlib import Path
from typing import List, Tuple
from tqdm import tqdm
import pandas as pd
from transformers import AutoTokenizer, AutoProcessor
from vllm import LLM, SamplingParams

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


INFO 09-15 11:19:29 [__init__.py:235] Automatically detected platform cuda.


In [2]:
# arguments
max_length = 128000 # can only be as long as your VRAM allows
model_path = "dummy_model"
docs_folder = "dummy_files/easy"
system_prompt = "You are a helpful assistant. You will be given a long context of concatenated documents with clues hidden in them."
question_list = "dummy_questions.csv"
output_file = "dummy_model.csv"

In [3]:
# supports thinking models
def extract_thinking_and_summary(text: str, bot: str = "<think>", eot: str = "</think>") -> Tuple[str, str]:
    if bot in text and eot not in text:
        return "", text
    if eot in text:
        if bot in text:
            return (
                text[text.index(bot) + len(bot):text.index(eot)].strip(),
                text[text.index(eot) + len(eot):].strip(),
            )
        else:
            return (
                text[:text.index(eot)].strip(),
                text[text.index(eot) + len(eot):].strip(),
            )
    return "", text

In [4]:
# load paths of documents (.txt files) in docs_folder
def load_documents(folder_path: str) -> List[Tuple[str, str]]:
    docs = []
    for file in sorted(Path(folder_path).glob("*.txt")):
        with open(file, "r", encoding="utf-8") as f:
            docs.append((file.name, f.read()))
            # print(docs[-1]) # for debugging
    return docs

In [5]:
# combine documents into one long context
def concat_documents(docs: List[Tuple[str, str]], tokenizer, max_tokens=max_length) -> Tuple[str, int]:
    combined_text = ""
    token_count = 0
    for name, text in docs:
        header = f"\n\n===== Document: {name} =====\n\n"
        combined_text += header + text
        tokens = tokenizer.encode(combined_text, add_special_tokens=False)
        token_count = len(tokens)
        print(f"Added {name}: cumulative tokens = {token_count}") # provides sensing of token contribution of each document
        if token_count > max_tokens:
            print(f"⚠️ Warning: Context size exceeded {max_tokens} tokens!") 
    return combined_text, token_count

In [6]:
def run_inference(model_path: str, docs_folder: str, system_prompt: str, question_list: str, output_file: str):
    start_time = time.time()

    # Load documents
    docs = load_documents(docs_folder)
    if not docs:
        print(f"No text files found in {docs_folder}.")
        return

    # Initialize tokenizer & processor
    tokenizer = AutoTokenizer.from_pretrained(model_path)
    processor = AutoProcessor.from_pretrained(model_path)

    # Concatenate documents with headers
    combined_text, total_tokens = concat_documents(docs, tokenizer)
    print(f"Concatenated context: {combined_text}")
    print(f"Total tokens in concatenated context: {total_tokens}")

    # Prepare questions from .csv file
    df = pd.read_csv(question_list)
    
    thoughts = []
    answers = []
    
    # Initialize vLLM model
    llm = LLM(
        model=model_path,
        trust_remote_code=True,
        tensor_parallel_size=1,
        gpu_memory_utilization=0.9,
        max_model_len=max_length
    )

    # Set sampling parameters
    sampling_params = SamplingParams(
        temperature=0.7,
        top_p=1,
        repetition_penalty=1.05,
        max_tokens=8196,
        stop_token_ids=[]
    )
    
    for i, row in tqdm(df.iterrows(), total=len(df), desc="Inference"):
        try:
            # Safe parsing of messages
            qn = row['question']
            messages = [{"role": "system", "content": system_prompt},
        {"role": "user", "content": combined_text + qn}]
            
            prompt = processor.apply_chat_template(
                messages,
                tokenize=False,
                add_generation_prompt=True,
            )
           
            llm_inputs = {"prompt": prompt}

            outputs = llm.generate([llm_inputs], sampling_params=sampling_params)
            generated_text = outputs[0].outputs[0].text

            thought, answer = extract_thinking_and_summary(generated_text)
            print("thought: ", thought)
            print("answer: ", answer)
            thoughts.append(thought)
            answers.append(answer)

        except Exception as e:
            print(f"[!] Error on row {i}: {e}")
            thoughts.append("")
            answers.append("")

    df["thoughts"] = thoughts
    df["answers"] = answers

    # Save result as .csv
    model_name = os.path.basename(model_path).replace("/", "_")
    output_path = Path(output_file or f"{model_name}_results.csv")
    df.to_csv(output_path)
    print(f"Results saved to {output_path}")
    elapsed = time.time() - start_time
    print(f"⏱️ Time taken: {elapsed:.2f} seconds")

Let's see if the model can extract 3 hidden clues in the concatenated text.

In [7]:
run_inference(model_path, docs_folder, system_prompt, question_list, output_file)

Added article1.txt: cumulative tokens = 1575
Added article2.txt: cumulative tokens = 2347
Added article3.txt: cumulative tokens = 3420
Added article4.txt: cumulative tokens = 3858
Added article5.txt: cumulative tokens = 4621
Added article6.txt: cumulative tokens = 5461
Added article7.txt: cumulative tokens = 6666
Added article8.txt: cumulative tokens = 7472
Added article9.txt: cumulative tokens = 8299
Concatenated context: 

===== Document: article1.txt =====

Pushing Boundaries in Flight: The 16th Singapore Amazing Flying Machine Competition Honours Innovation with Thrilling Demonstrations and a Celebration of Creativity

Singapore, 5 April 2025 – The Awards Ceremony for the 16th Singapore Amazing Flying Machine Competition (SAFMC) 2025 took place today at the Singapore University of Technology and Design (SUTD). Co-organised by DSO National Laboratories (DSO) and Science Centre Singapore (SCS), this year’s competition brought together 1,865 participants across 584 teams, competing in

2025-09-15 11:19:31,973	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
`torch_dtype` is deprecated! Use `dtype` instead!


INFO 09-15 11:19:38 [config.py:1604] Using max model len 128000
INFO 09-15 11:19:38 [config.py:2434] Chunked prefill is enabled with max_num_batched_tokens=8192.
INFO 09-15 11:19:43 [__init__.py:235] Automatically detected platform cuda.
INFO 09-15 11:19:45 [core.py:572] Waiting for init message from front-end.
INFO 09-15 11:19:45 [core.py:71] Initializing a V1 LLM engine (v0.10.0) with config: model='/home/jovyan/dummy_model', speculative_config=None, tokenizer='/home/jovyan/dummy_model', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config={}, tokenizer_revision=None, trust_remote_code=True, dtype=torch.bfloat16, max_seq_len=128000, download_dir=None, load_format=auto, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto,  device_config=cuda, decoding_config=DecodingConfig(backend='auto', disable_fallback=False, disable_any_whitespace=False, disable_additional_pr

Loading safetensors checkpoint shards:   0% Completed | 0/2 [00:00<?, ?it/s]
Loading safetensors checkpoint shards:  50% Completed | 1/2 [00:01<00:01,  1.79s/it]
Loading safetensors checkpoint shards: 100% Completed | 2/2 [00:03<00:00,  1.45s/it]
Loading safetensors checkpoint shards: 100% Completed | 2/2 [00:03<00:00,  1.50s/it]



INFO 09-15 11:19:51 [default_loader.py:262] Loading weights took 3.08 seconds
INFO 09-15 11:19:51 [gpu_model_runner.py:1892] Model loading took 7.1694 GiB and 3.266711 seconds
INFO 09-15 11:19:56 [backends.py:530] Using cache directory: /home/jovyan/.cache/vllm/torch_compile_cache/60cf5f0370/rank_0_0/backbone for vLLM's torch.compile
INFO 09-15 11:19:56 [backends.py:541] Dynamo bytecode transform time: 4.91 s
INFO 09-15 11:20:03 [backends.py:161] Directly load the compiled graph(s) for dynamic shape from the cache, took 6.583 s
INFO 09-15 11:20:04 [monitor.py:34] torch.compile takes 4.91 s in total
INFO 09-15 11:20:05 [gpu_worker.py:255] Available KV cache memory: 30.86 GiB
INFO 09-15 11:20:05 [kv_cache_utils.py:833] GPU KV cache size: 252,784 tokens
INFO 09-15 11:20:05 [kv_cache_utils.py:837] Maximum concurrency for 128,000 tokens per request: 1.97x


...


Adding requests: 100%|██████████| 1/1 [00:00<00:00, 46.03it/s]

Inference:  67%|██████▋   | 2/3 [00:01<00:00,  1.63it/s]

thought:  
answer:  The second clue provided in the text is "DSO53."



Adding requests: 100%|██████████| 1/1 [00:00<00:00, 45.44it/s]

Inference: 100%|██████████| 3/3 [00:01<00:00,  1.84it/s]


thought:  
answer:  Clue #3 is "I love IEL!"
Results saved to dummy_model.csv
⏱️ Time taken: 41.11 seconds


[rank0]:[W915 11:20:12.246120972 ProcessGroupNCCL.cpp:1479] Warning: WARNING: destroy_process_group() was not called before program exit, which can leak resources. For more info, please see https://pytorch.org/docs/stable/distributed.html#shutdown (function operator())


It seems like a context length of 8k isn't that big of a problem for the model. What if we extend the context length with more documents?

In [8]:
pd.read_csv("dummy_model.csv")[["question", "answers"]]

,question,answers
0,What does the first clue say?,"The first clue says ""People Passion Innovation."""
1,What is the 2nd clue?,"The second clue provided in the text is ""DSO53."""
2,What is Clue #3?,"Clue #3 is ""I love IEL!"""


In [9]:
run_inference(model_path, "dummy_files/hard", system_prompt, question_list, output_file)

Added article1.txt: cumulative tokens = 1575
Added article2.txt: cumulative tokens = 2347
Added article3.txt: cumulative tokens = 3420
Added article4.txt: cumulative tokens = 3858
Added article5.txt: cumulative tokens = 4621
Added article6.txt: cumulative tokens = 5461
Added article7.txt: cumulative tokens = 6666
Added article8.txt: cumulative tokens = 7472
Added article9.txt: cumulative tokens = 8299
Added paper1.txt: cumulative tokens = 25430
Added paper2.txt: cumulative tokens = 42257
Concatenated context: 

===== Document: article1.txt =====

Pushing Boundaries in Flight: The 16th Singapore Amazing Flying Machine Competition Honours Innovation with Thrilling Demonstrations and a Celebration of Creativity

Singapore, 5 April 2025 – The Awards Ceremony for the 16th Singapore Amazing Flying Machine Competition (SAFMC) 2025 took place today at the Singapore University of Technology and Design (SUTD). Co-organised by DSO National Laboratories (DSO) and Science Centre Singapore (SCS), th

Loading safetensors checkpoint shards:   0% Completed | 0/2 [00:00<?, ?it/s]
Loading safetensors checkpoint shards:  50% Completed | 1/2 [00:01<00:01,  1.75s/it]
Loading safetensors checkpoint shards: 100% Completed | 2/2 [00:02<00:00,  1.42s/it]
Loading safetensors checkpoint shards: 100% Completed | 2/2 [00:02<00:00,  1.47s/it]



INFO 09-15 11:20:27 [default_loader.py:262] Loading weights took 3.01 seconds
INFO 09-15 11:20:27 [gpu_model_runner.py:1892] Model loading took 7.1694 GiB and 3.215795 seconds
INFO 09-15 11:20:32 [backends.py:530] Using cache directory: /home/jovyan/.cache/vllm/torch_compile_cache/60cf5f0370/rank_0_0/backbone for vLLM's torch.compile
INFO 09-15 11:20:32 [backends.py:541] Dynamo bytecode transform time: 4.95 s
INFO 09-15 11:20:39 [backends.py:161] Directly load the compiled graph(s) for dynamic shape from the cache, took 6.553 s
INFO 09-15 11:20:40 [monitor.py:34] torch.compile takes 4.95 s in total
INFO 09-15 11:20:41 [gpu_worker.py:255] Available KV cache memory: 30.86 GiB
INFO 09-15 11:20:42 [kv_cache_utils.py:833] GPU KV cache size: 252,784 tokens
INFO 09-15 11:20:42 [kv_cache_utils.py:837] Maximum concurrency for 128,000 tokens per request: 1.97x


Capturing CUDA graph shapes: 100%|██████████| 67/67 [00:03<00:00, 22.13it/s]


INFO 09-15 11:20:45 [gpu_model_runner.py:2485] Graph capturing finished in 3 secs, took 0.57 GiB
INFO 09-15 11:20:45 [core.py:193] init engine (profile, create kv cache, warmup model) took 17.75 seconds


...


Adding requests: 100%|██████████| 1/1 [00:00<00:00,  8.74it/s]

Inference:  67%|██████▋   | 2/3 [00:08<00:03,  3.52s/it]

thought:  
answer:  The second clue is: "People Passion Innovation."



Adding requests: 100%|██████████| 1/1 [00:00<00:00,  7.60it/s]

Inference: 100%|██████████| 3/3 [00:21<00:00,  7.13s/it]


thought:  
answer:  Clue #3 is hidden within the description of DSO National Laboratories and Science Centre Singapore, mentioning their roles and capabilities. The phrase "People Passion Innovation" appears as part of the organization's profile, hinting at a name or slogan. By examining the provided excerpts, we can't find a direct mention of a slogan or motto containing exactly "People Passion Innovation." However, looking at the structure of the documents, we notice repeated mentions of DSO and SCS, possibly representing the acronym or initials for their full titles, Department of Scientific Organizations and Centers of Science and Safety.

Given the format of the articles and the hints provided, it's plausible that "People Passion Innovation" is meant to symbolize something related to the departments involved in organizing the Singapore Amazing Flying Machine Competition, perhaps as their catchphrase or theme. But strictly speaking, none of the texts explicitly confirm this phrase 

[rank0]:[W915 11:21:08.415402203 ProcessGroupNCCL.cpp:1479] Warning: WARNING: destroy_process_group() was not called before program exit, which can leak resources. For more info, please see https://pytorch.org/docs/stable/distributed.html#shutdown (function operator())


Interesting... the model forgot what the second and third clue were! Can you explain why, and when hallucinations occur?

In [10]:
pd.read_csv("dummy_model.csv")[["question", "answers"]]

,question,answers
0,What does the first clue say?,"The first clue says: ""People Passion Innovation"""
1,What is the 2nd clue?,"The second clue is: ""People Passion Innovation."""
2,What is Clue #3?,Clue #3 is hidden within the description of DS...
